In [1]:
import simpy
from numpy.random import choice
#from random import expovariate
import numpy as np

In [2]:
n=5 #Prime router number
N=10 #regular router number
m=3 #shortest path number

In [3]:
file_name_shortest_path='full_graph_shortest_path_{}_{}.txt'.format(n,N)
file_name_prime_graph='prime_graph_{}_{}.txt'.format(n,N)
file_name_prime_graph_optimal_path='prime_graph_optimal_path_{}_{}.txt'.format(n,N)
file_best_clustering='full_graph_best_clustering_{}_{}.txt'.format(n,N)
file_name_cluster_graph='cluster_graph_{}_{}.txt'.format(n,N)

# Reading clustering and prime graphs

In [4]:
con=[[] for _ in range(n+N+1)]
edges_clust=set()

In [5]:
with open('test/'+file_name_cluster_graph, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    #f.write('\n')
    for line in f:
        #print('line',line)
        x,y,w=map(float,line.split())
        x,y=int(x),int(y)
        #print(x,y,w)
        con[x].append((y,w))
        #con[y].append((x,w))
        edges_clust.add((x,y,w))

           
    f.close()

In [6]:
edges_clust

{(1, 3, 0.23142470169810703),
 (1, 5, 0.7536451376844908),
 (1, 6, 0.4638092727662523),
 (1, 7, 0.9581887310477116),
 (1, 8, 0.11962100470551662),
 (1, 13, 0.10916629732677441),
 (1, 14, 0.9991049222877524),
 (2, 3, 0.032934312265980425),
 (2, 5, 0.6369470115993228),
 (2, 6, 0.9557802366404332),
 (2, 9, 0.3430049053278973),
 (2, 14, 0.7731763192800118),
 (3, 1, 0.23142470169810703),
 (3, 2, 0.032934312265980425),
 (3, 4, 0.501869175806194),
 (3, 5, 0.882580654303218),
 (3, 6, 0.013962574310672937),
 (3, 7, 0.3114343887508788),
 (3, 11, 0.6107640985517377),
 (3, 12, 0.7868252034308975),
 (3, 14, 0.9658427179915304),
 (4, 3, 0.501869175806194),
 (4, 6, 0.7856803587257708),
 (4, 8, 0.6293629338431845),
 (4, 9, 0.15686755357253557),
 (4, 13, 0.959083650317909),
 (5, 1, 0.7536451376844908),
 (5, 2, 0.6369470115993228),
 (5, 3, 0.882580654303218),
 (5, 11, 0.527469403352322),
 (5, 12, 0.9106297271042973),
 (5, 13, 0.2205953804350782),
 (6, 1, 0.4638092727662523),
 (6, 2, 0.9557802366404332),

In [7]:
con=[[] for _ in range(n+N+1)]
edges_prime=set()

In [8]:
with open('test/'+file_name_prime_graph, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    #f.write('\n')
    for line in f:
        #print('line',line)
        x,y,w=map(float,line.split())
        x,y=int(x),int(y)
        #print(x,y,w)
        con[x+N].append((y+N,w))
        #con[y].append((x,w))
        edges_prime.add((x+N,y+N,w))

           
    f.close()

In [9]:
edges_prime

{(11, 12, 6.089308538078741),
 (12, 11, 6.089308538078741),
 (12, 13, 3.0),
 (12, 14, 4.977825363992857),
 (12, 15, 5.3342404165033095),
 (13, 12, 3.0),
 (13, 15, 3.1448932613183276),
 (14, 12, 4.977825363992857),
 (15, 12, 5.3342404165033095),
 (15, 13, 3.1448932613183276)}

In [10]:
edges=edges_clust.union(edges_prime)

In [11]:
edges

{(1, 3, 0.23142470169810703),
 (1, 5, 0.7536451376844908),
 (1, 6, 0.4638092727662523),
 (1, 7, 0.9581887310477116),
 (1, 8, 0.11962100470551662),
 (1, 13, 0.10916629732677441),
 (1, 14, 0.9991049222877524),
 (2, 3, 0.032934312265980425),
 (2, 5, 0.6369470115993228),
 (2, 6, 0.9557802366404332),
 (2, 9, 0.3430049053278973),
 (2, 14, 0.7731763192800118),
 (3, 1, 0.23142470169810703),
 (3, 2, 0.032934312265980425),
 (3, 4, 0.501869175806194),
 (3, 5, 0.882580654303218),
 (3, 6, 0.013962574310672937),
 (3, 7, 0.3114343887508788),
 (3, 11, 0.6107640985517377),
 (3, 12, 0.7868252034308975),
 (3, 14, 0.9658427179915304),
 (4, 3, 0.501869175806194),
 (4, 6, 0.7856803587257708),
 (4, 8, 0.6293629338431845),
 (4, 9, 0.15686755357253557),
 (4, 13, 0.959083650317909),
 (5, 1, 0.7536451376844908),
 (5, 2, 0.6369470115993228),
 (5, 3, 0.882580654303218),
 (5, 11, 0.527469403352322),
 (5, 12, 0.9106297271042973),
 (5, 13, 0.2205953804350782),
 (6, 1, 0.4638092727662523),
 (6, 2, 0.9557802366404332),

# Reading optimal clustering

In [12]:
clust=[0]*(N+1)

reverse_clust=[set() for i in range(n+N+1)] 

clust_opt_path=[0]*(N+1)

In [13]:
with open('test/'+file_best_clustering, 'r') as f:
    
    n,N=map(int,f.readline().split())
    next(f)
    
    i=1
    line_cnt=0
    for line in f:
        line_cnt+=1 
        
        if line_cnt%3:
            if line_cnt%3==1:
                i,j=map(int,line.split())
                clust[i]=j
                reverse_clust[j].add(i)
            else:
                
                path = list(map(int,line.split()))
                
                #print('path',path)
                clust_opt_path[i]=path
                i+=1
    f.close()

In [14]:
n

5

In [15]:
clust[1:]

[15, 14, 14, 12, 12, 12, 14, 13, 11, 14]

In [16]:
reverse_clust[N+1:]

[{9}, {4, 5, 6}, {8}, {2, 3, 7, 10}, {1}]

In [17]:
clust_opt_path[1:]

[[1, 15],
 [2, 9, 7, 14],
 [3, 14],
 [4, 12],
 [5, 2, 12],
 [6, 12],
 [7, 14],
 [8, 7, 9, 13],
 [9, 11],
 [10, 8, 7, 14]]

# Reading optimal routing from prime graph

In [18]:
prime_opt_path=[0]*(n+1)

for i in range(n+1):
    prime_opt_path[i]=[0]*(n+1)

In [19]:
with open('test/'+file_name_prime_graph_optimal_path, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    line_cnt=0
    for line in f:
        line_cnt+=1 
        
        if line_cnt%3:
            
            if line_cnt%3==1:
                #print('line',line)
                i,j=map(int,line.split())
            else:
                path = list(map(int,line.split()))
                #print('path',path)

                prime_opt_path[i][j]=np.array(path)+N
        
    f.close()

In [20]:
prime_opt_path

[[0, 0, 0, 0, 0, 0],
 [0,
  0,
  array([11, 12]),
  array([11, 12, 13]),
  array([11, 12, 14]),
  array([11, 12, 13, 15])],
 [0,
  array([12, 11]),
  0,
  array([12, 13]),
  array([12, 14]),
  array([12, 13, 15])],
 [0,
  array([13, 15, 12, 11]),
  array([13, 12]),
  0,
  array([13, 15, 12, 14]),
  array([13, 12, 15])],
 [0,
  array([14, 12, 11]),
  array([14, 12]),
  array([14, 12, 15, 13]),
  0,
  array([14, 12, 13, 15])],
 [0,
  array([15, 12, 11]),
  array([15, 13, 12]),
  array([15, 13]),
  array([15, 13, 12, 14]),
  0]]

In [21]:
clust[1]

15

In [22]:
clust[2]

14

In [23]:
clust_opt_path

[0,
 [1, 15],
 [2, 9, 7, 14],
 [3, 14],
 [4, 12],
 [5, 2, 12],
 [6, 12],
 [7, 14],
 [8, 7, 9, 13],
 [9, 11],
 [10, 8, 7, 14]]

# Optimal routing in full graph

In [24]:
full_graph_opt_path=[0]*(N+1)

for i in range(1,N+1):
    full_graph_opt_path[i]=[0]*(N+1)

In [25]:
clust_opt_path[1]

[1, 15]

In [26]:
clust_opt_path[2]

[2, 9, 7, 14]

In [27]:
prime_opt_path[4][5]

array([14, 12, 13, 15])

In [28]:
prime_opt_path[2][3]

array([12, 13])

In [29]:
for i in range(1,N+1):
    for j in range(1,N+1):
        if i != j:
            prime_i=clust[i]
            prime_j=clust[j]
            
            if prime_i==prime_j:
                full_graph_opt_path[i][j]=[]
                continue
            
            #print('hhh',i,j,prime_i,prime_j)
            
            prime_path=prime_opt_path[prime_i-N][prime_j-N]
            #print(i,j, 'prime_path!',prime_path)
            
            l=len(prime_path)
            #print(i,j,clust_opt_path[i],prime_path[1:l-1],clust_opt_path[j][::-1])
            
            
            full_graph_opt_path[i][j]=clust_opt_path[i]+list(prime_path[1:l-1])+clust_opt_path[j][::-1]
            
            

# Network simulation

In [30]:
class Packet(object):
    """ A very simple class that represents a packet.
        This packet will run through a queue at a switch output port.
        We use a float to represent the size of the packet in bytes so that
        we can compare to ideal M/M/1 queues.

        Parameters
        ----------
        
        packet_id : int
            an identifier for the packet
            
        path : list
            path from source to destination
            
        cur_path_ind : int
            current path position
        
        time : float
            the time the packet arrives at the output queue
            
        size : float
            the size of the packet in bytes
        
        src, dst : int
            identifiers for the source and the destination

    """
    def __init__(self, packet_id, path, cur_path_ind, time, size, src, dst):
        
        self.packet_id=packet_id
        self.path=path
        self.cur_path_ind=cur_path_ind
        
        
        self.time = time
        self.size = size
        self.src = src
        self.dst = dst


    def __repr__(self):
        return "id: {}, src: {}, time: {}, size: {}".\
            format(self.packet_id, self.src, self.time, self.size)

In [31]:
class PacketGenerator(object):
    """ Generates packets with given inter-arrival time distribution. 
        Set the "out" member variable to the entity to receive the packet.
        Then it generates a destination (for each package independently)
        according to predetermined distribution and sends packages along the predetemined paths.
        PacketGenerator i is connected to Switch i.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
            
        gen_prob : list
            destination probability distribution
            
        generator_id : int
            PacketGenerator id
            
        adist : function
            a no parameter function that returns the successive inter-arrival times of the packets
            
        sdist : function
            a no parameter function that returns the successive sizes of the packets
            
        initial_delay : int
            Starts generation after an initial delay. Default = 0
            
        finish : int
            Stops generation at the finish time. Default is infinite


    """
    def __init__(self, env, gen_prob, generator_id, adist, sdist, initial_delay=0, finish=float("inf")):
        
        self.gen_prob=gen_prob
        self.generator_id = generator_id
        self.env = env
        self.adist = adist
        self.sdist = sdist
        self.initial_delay = initial_delay
        self.finish = finish
        self.packets_sent = 0
        self.action = env.process(self.run())  # starts the run() method as a SimPy process


    def run(self):
        """The generator function used in simulations.
        """
        
        yield self.env.timeout(self.initial_delay)
        while self.env.now < self.finish:
            yield self.env.timeout(self.adist())
            
            destination = choice(range(1,N+1), p=self.gen_prob)
            opt_path=full_graph_opt_path[self.generator_id][destination]

            self.packets_sent += 1
            
            pkt = Packet(packet_id=self.packets_sent, 
                         path=opt_path, 
                         cur_path_ind=0, 
                         time=self.env.now, 
                         size=self.sdist(), 
                         src=self.generator_id, 
                         dst=destination)

            
            self.out.put(pkt)
       

In [32]:
class PacketSink(object):
    """ Receives packets and collects delay information into the
        waits list. You can then use this list to look at delay statistics.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
            
        debug : boolean
            if true then the contents of each packet will be printed as it is received.
            
        rec_arrivals : boolean
            if true then arrivals will be recorded
            
        absolute_arrivals : boolean
            if true absolute arrival times will be recorded, otherwise the time between consecutive arrivals
            is recorded.
            
        rec_waits : boolean
            if true waiting time experienced by each packet is recorded
            

    """
    def __init__(self, env, rec_arrivals=False, absolute_arrivals=False, rec_waits=True, debug=False):
        self.store = simpy.Store(env)
        self.env = env
        self.rec_waits = rec_waits
        self.rec_arrivals = rec_arrivals
        self.absolute_arrivals = absolute_arrivals
        self.waits = []
        self.arrivals = []
        self.debug = debug
        self.packets_rec = 0
        self.bytes_rec = 0
        self.last_arrival = 0.0

    def put(self, pkt):

        now = self.env.now
        
        if self.rec_waits:
            self.waits.append(self.env.now - pkt.time)
        if self.rec_arrivals:
            if self.absolute_arrivals:
                self.arrivals.append(now)
            else:
                self.arrivals.append(now - self.last_arrival)
            self.last_arrival = now
        self.packets_rec += 1
        self.bytes_rec += pkt.size


In [33]:
class SwitchPort(object):
    """ Models a switch output port with a given rate and buffer size limit in bytes.
        First, it stores packets in queue (put method), then it extracts them one-by-one 
        and sends to the next switch or sink.
        

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        rate : float
            the bit rate of the port
        qlimit : integer (or None)
            a buffer size limit in bytes or packets for the queue (including items
            in service).
        limit_bytes : If true, the queue limit will be based on bytes if false the
            queue limit will be based on packets.

    """
    def __init__(self, env, router_id, rate, qlimit=None, limit_bytes=True, debug=False):
        self.router_id=router_id
        self.store = simpy.Store(env)
        self.rate = rate
        self.env = env
        self.out = None
        self.packets_rec = 0
        self.packets_drop = 0
        self.qlimit = qlimit
        self.limit_bytes = limit_bytes
        self.byte_size = 0  # Current size of the queue in bytes
        self.debug = debug
        self.busy = 0  # Used to track if a packet is currently being sent
        self.action = env.process(self.run())  # starts the run() method as a SimPy process

    def run(self):

        while True:
            msg = (yield self.store.get())
            self.busy = 1
            self.byte_size -= msg.size
            yield self.env.timeout(msg.size*8.0/self.rate)
            
            path=msg.path
            cur_path_ind=msg.cur_path_ind
            destination=msg.dst
                
                
            if path[cur_path_ind]==destination:
                
                sink.put(msg)
            else:
                next_node=path[cur_path_ind+1]
                msg.cur_path_ind=cur_path_ind+1
                
                #print('',)
                #print('next_node',next_node)
                #print('switch[next_node]',switch[next_node])
                switch[next_node].store.put(msg)   
                
                
                
            self.busy = 0
            if self.debug:
                print(msg)
                
     

    def put(self, pkt):

        self.packets_rec += 1
        tmp_byte_count = self.byte_size + pkt.size

        if self.limit_bytes and tmp_byte_count >= self.qlimit:
            self.packets_drop += 1
            return
        
        else:
            self.byte_size = tmp_byte_count
            
            return self.store.put(pkt)    
                

In [34]:
def RandomizedArrival():
    #return 1.5    # time interval
    return max(np.random.normal(1.5, 0.5),1)

def constSize():
    return 100.0  # bytes

In [35]:

prob=[0]*4
for i in range(4):
    path[i]=[0]*4



gen_prob=[0]*4
gen_prob[1]=[0,0.5,0.5]
gen_prob[2]=[0.2,0,0.8]
gen_prob[3]=[0.7,0.3,0]

In [36]:
gen_prob=[0]*(N+1)

for i in range(1,N+1):
    prime_router=clust[i]
    #print('reverse_clust(prime_router)',reverse_clust[prime_router])
    clust_len=len(reverse_clust[prime_router])
    
    gen_prob[i]=[1./(N-clust_len) if not j+1 in reverse_clust[prime_router] else 0 for j in range(N)]
    


In [37]:
gen_prob

[0,
 [0,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111],
 [0.16666666666666666,
  0,
  0,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0,
  0.16666666666666666,
  0.16666666666666666,
  0],
 [0.16666666666666666,
  0,
  0,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0,
  0.16666666666666666,
  0.16666666666666666,
  0],
 [0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0,
  0,
  0,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285],
 [0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0,
  0,
  0,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285],
 [0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0,
  0,
  0,
  0.14285714285714285,
  0.142857142

In [49]:
generators=[0]*(N+1)
switch=[0]*(n+N+1)

env = simpy.Environment()  # Create the SimPy environment

sink=PacketSink(env, debug=True)

for i in range(1,n+N+1):
    switch[i]=SwitchPort(env=env, router_id=i, rate=1000.0, qlimit=3000, debug=True)

for i in range(1,N+1):
    generators[i]=PacketGenerator(env, gen_prob[i], i, RandomizedArrival, constSize)
    generators[i].out = switch[i]


    
env.run(until=200)
print("average delivery time:", format(sum(sink.waits)/max(1,sink.packets_rec)))
print('number of delivered packages:', sink.packets_rec)
print('number of packages stuck in the network:\n')

for i in range(1,n+N+1):
    print('router {} packages {}'.format(i,len(switch[i].store.items)))

print('\n')
for i in range(1,n+N+1):
    print("router {} has dropped {} packages ".format(i,switch[i].packets_drop))

print('\n')
for i in range(1,N+1):        
    print('generator {} has sent {} packages'.format(i, generators[i].packets_sent))


id: 1, src: 4, time: 1, size: 100.0
id: 1, src: 9, time: 1, size: 100.0
id: 1, src: 1, time: 1.1156307114481874, size: 100.0
id: 1, src: 6, time: 1.1381938127562485, size: 100.0
id: 1, src: 5, time: 1.2341480581580895, size: 100.0
id: 1, src: 8, time: 1.6857371498994642, size: 100.0
id: 1, src: 2, time: 1.758125766625573, size: 100.0
id: 1, src: 3, time: 1.7734515601511378, size: 100.0
id: 1, src: 4, time: 1, size: 100.0
id: 1, src: 9, time: 1, size: 100.0
id: 1, src: 10, time: 1.8705377473405744, size: 100.0
id: 1, src: 1, time: 1.1156307114481874, size: 100.0
id: 2, src: 1, time: 2.1156307114481874, size: 100.0
id: 2, src: 6, time: 2.2206912140239945, size: 100.0
id: 1, src: 7, time: 2.402900786512829, size: 100.0
id: 2, src: 9, time: 2.519090861953332, size: 100.0
id: 1, src: 5, time: 1.2341480581580895, size: 100.0
id: 1, src: 3, time: 1.7734515601511378, size: 100.0
id: 1, src: 6, time: 1.1381938127562485, size: 100.0
id: 1, src: 4, time: 1, size: 100.0
id: 1, src: 10, time: 1.870

id: 6, src: 8, time: 7.66573968279081, size: 100.0
id: 42, src: 3, time: 65.74009002777417, size: 100.0
id: 53, src: 8, time: 79.78565441002408, size: 100.0
id: 52, src: 5, time: 82.73654195161895, size: 100.0
id: 62, src: 10, time: 95.80271992509967, size: 100.0
id: 60, src: 1, time: 91.57904293371364, size: 100.0
id: 15, src: 3, time: 23.602120796274683, size: 100.0
id: 8, src: 7, time: 12.90643688062363, size: 100.0
id: 61, src: 5, time: 96.03515876806698, size: 100.0
id: 8, src: 4, time: 14.04496993321571, size: 100.0
id: 13, src: 4, time: 21.694783053993817, size: 100.0
id: 61, src: 4, time: 96.14339809756262, size: 100.0
id: 21, src: 4, time: 34.057759569436435, size: 100.0
id: 17, src: 6, time: 24.174740015177317, size: 100.0
id: 52, src: 10, time: 79.42208689783388, size: 100.0
id: 57, src: 2, time: 84.54054734333683, size: 100.0
id: 64, src: 1, time: 96.5032280721585, size: 100.0
id: 61, src: 1, time: 92.88088753022205, size: 100.0
id: 63, src: 1, time: 95.16216744524377, size

id: 110, src: 8, time: 163.9123622242284, size: 100.0
id: 109, src: 2, time: 167.15320797590783, size: 100.0
id: 123, src: 6, time: 190.9292394737244, size: 100.0
id: 124, src: 10, time: 191.0036152934466, size: 100.0
id: 124, src: 1, time: 190.26149619537836, size: 100.0
id: 11, src: 10, time: 17.11056649873545, size: 100.0
id: 56, src: 8, time: 84.10748636281286, size: 100.0
id: 76, src: 9, time: 123.22596779127528, size: 100.0
id: 29, src: 3, time: 44.976411984952364, size: 100.0
id: 82, src: 3, time: 128.49853512734956, size: 100.0
id: 23, src: 5, time: 37.4311062116727, size: 100.0
id: 104, src: 10, time: 163.63768576539735, size: 100.0
id: 110, src: 2, time: 168.15320797590783, size: 100.0
id: 123, src: 5, time: 191.7015038456869, size: 100.0
id: 124, src: 1, time: 190.26149619537836, size: 100.0
id: 124, src: 6, time: 191.94845362096132, size: 100.0
id: 66, src: 7, time: 102.61416366165095, size: 100.0
id: 71, src: 2, time: 109.09827603995504, size: 100.0
id: 76, src: 9, time: 1